# Evaluation YOLO model

## Libraries

In [ ]:
import os
import tempfile
import yaml
from ultralytics import YOLO

## Evaluation on a folder

In [ ]:
def evaluate_model(weights_path: str,
                   images_path: str,
                   labels_path: str,
                   class_names: dict,
                   iou_threshold: float = 0.6):
    """
    Validates a YOLO model using a temporary YAML file generated on the fly.

    weights_path : path to the .pt model
    images_path  : folder containing test images        (…/dataset/images/test)
    labels_path  : folder containing test .txt labels   (…/dataset/labels/test)
    class_names  : {id: class_name}
    """

    # ---------- 1) Basic checks ----------
    if not os.path.isdir(images_path):
        raise FileNotFoundError(f"Image folder does not exist: {images_path}")
    if not os.path.isdir(labels_path):
        raise FileNotFoundError(f"Label folder does not exist: {labels_path}")

    # ---------- 2) Dataset root ----------
    # (two levels above the images_path → …/dataset)
    dataset_root = os.path.abspath(os.path.join(images_path, os.pardir, os.pardir))

    # ---------- 3) YAML construction ----------
    # Ultralytics only needs the image folders. It automatically looks for the .txt files in 'labels/'.
    data_yaml = {
        "path": dataset_root.replace("\\", "/"),   # absolute path, no backslashes
        "train": "images/train",                   # ✓ must exist even if unused (can be empty)
        "val":   "images/test",
        "test":  "images/test",
        "names": class_names
    }

    # ---------- 4) Save temporary YAML ----------
    with tempfile.NamedTemporaryFile(mode="w", suffix=".yaml", delete=False) as f:
        yaml.dump(data_yaml, f, sort_keys=False)
        tmp_yaml_path = f.name

    try:
        # ---------- 5) Validation ----------
        model = YOLO(weights_path)
        results = model.val(data=tmp_yaml_path, iou=iou_threshold)

    finally:
        # ---------- 6) Cleanup ----------
        os.remove(tmp_yaml_path)


# ----------------------------- USAGE EXAMPLE -----------------------------

WEIGHTS     = "model.pt"
IMAGES_TEST = "dataset/images/test"
LABELS_TEST = "dataset/labels/test"
CLASS_NAMES = {
    0: "class_0",
    1: "class_1",
    2: "class_2",
    3: "class_3",
    4: "class_4",
    5: "class_5",
    6: "class_6",
    7: "class_7",
    8: "class_8",
    9: "class_9",
    }


#-------------------------------------------------------------------------
# Generates a folder called runs/val/ in the current working directory
# containing the results of the validation.
evaluate_model(WEIGHTS, IMAGES_TEST, LABELS_TEST, CLASS_NAMES)
